In [3]:
import serial

In [119]:
import glob


['/dev/ttyUSB1']


In [166]:
class DldSimulatorException(RuntimeError):
    def __init__(self, arg):
        self.args = arg

class PulseConfig:
    t0     =  100
    x      =    0
    y      =    0
    active = True
    
    def validate(self):
        valid = True
        if (self.t0 < 0):
            valid = False
        if (self.t0 > 800):
            valid = False
        if (self.x < -32):
            valid = False
        if (self.x > 31):
            valid = False
        if (self.y < -32):
            valid = False
        if (self.y > 31):
            valid = False
        return valid
            
    def createControlWord(self):
        if (not self.validate()):
            raise DldSimulatorException("Invalid pulse configuration")
        controlWord = self.t0
        controlWord = controlWord | (((self.x + 32) & 0b111111) << 10)
        controlWord = controlWord | (((self.y + 32) & 0b111111) << 16)
        if (self.active):
            controlWord = controlWord | (0x1 << 22)
        return controlWord
    
    def readControlWord(self, controlWord):
        self.t0 = controlWord & 0b1111111111
        self.x  = ((controlWord >> 10) & 0b111111) -32
        self.y  = ((controlWord >> 16) & 0b111111) -32
        self.active = (((controlWord >> 22) & 0x01) == 1)
        if (not self.validate()):
            raise DldSimulatorException("Invalid pulse configuration")
        
    def to_string(self):
        return "t0 = " + str(self.t0) + "  x = " + str(self.x) + "  y = " + str(self.y) + " active = " + str(self.active)
    
    def set(self, t0, x, y, active):
        self.x = x
        self.y = y
        self.t0 = t0
        self.active = active
        




    
class DldSimulator:
    
    ADDR_START_CONFIG       = 0x44a0_0004
    ADDR_MULTI_SPECTRUM_GEN = 0x44a0_0100
    ADDR_LIST_GEN1_BASE     = 0x44a0_0200
    ADDR_LIST_GEN2_BASE     = 0x44a0_0400
    
    
    def __init__(self, commPort):
        self.s = serial.Serial(commPort, baudrate=115200)
        self.s.timeout = 1.0
        self.s.reset_input_buffer()
        self.s.reset_output_buffer()
        
        
    def __delete__(self):
        self.s.close()
        

    """
    Write one word (32 bits) to the data bus
    """
    def writeToBus(self, addr, data):
        send_str = "w " + hex(addr) + " " + hex(data) + "\n"
        self.s.write(send_str.encode('ascii'))
        ret = self.s.readline()
        if (ret.decode('ascii') != 'OK\n'):
            raise DldSimulatorException("Communication error")

    """
    Read one word (32 bits) from the data bus
    """
    def readFromBus(self, addr):
        send_str = "r " + hex(addr) + "\n"
        self.s.write(send_str.encode('ascii'))
        ret = self.s.readline().decode('ascii')
        return int(ret, 16)

    """
    Use the external start signal. The start interval settings are unaltered.
    """
    def useExternalStart(self):
        oldValue = self.readFromBus(self.ADDR_START_CONFIG)
        self.writeToBus(self.ADDR_START_CONFIG, oldValue | 0x0001_0000)

    """
    Use the internally generated start signal. The start interval settings are unaltered.
    """    
    def useInternalStart(self):
        oldValue = self.readFromBus(self.ADDR_START_CONFIG)
        self.writeToBus(self.ADDR_START_CONFIG, oldValue & ~0x0001_0000)
    
    """
    Return True if the external start signal is used.
    """
    def isExternalStart(self):
        value = self.readFromBus(self.ADDR_START_CONFIG)
        return ((value & 0x0001_0000) == 0x0001_0000)

    """
    Set the start interval (num. of clock cycles between internally generated start pulses)
    This method does not switch to the internally generated start signal.
    """
    def setInternalStartInterval(self,interval):
        if (interval < 100):
            raise DldSimulatorException("Start inverval too small")
        if (interval > 0xffff):
            raise DldSimulatorException("Start inverval too large")
        oldValue = self.readFromBus(self.ADDR_START_CONFIG)
        self.writeToBus(self.ADDR_START_CONFIG, (oldValue & 0xffff_0000) | interval)
    
    
    """
    Get the start interval (num. of clock cycles between internally generated start pulses)
    """
    def getInternalStartInterval(self):
        value = self.readFromBus(self.ADDR_START_CONFIG)
        return (value & 0xffff)

    """
    Activate numOfGenerators spectrum generators
    """
    def setActiveSpectrumGenerators(self, numOfGenerators):
        if (numOfGenerators < 0):
            raise DldSimulatorException("Num of generators can not be negative")
        if (numOfGenerators > 32):
            raise DldSimulatorException("Num of generators can not be larger than 32")
        value = 0xffffffff >> (32-numOfGenerators)
        self.writeToBus(self.ADDR_MULTI_SPECTRUM_GEN, value)
        
    """
    Get the number of active spectrum generators
    """    
    def getActiveSpectrumGenerators(self):
        value = self.readFromBus(self.ADDR_MULTI_SPECTRUM_GEN)
        return bin(value).count("1")
    
    
    def setListGenerator(self, channel, listOfPulses):
        if (channel < 1):
            raise DldSimulatorException("Channel number can not be negative")
        if (channel > 2):
            raise DldSimulatorException("Channel number can not be larger than 1")
        
        numOfPulses = len(listOfPulses)
        
        if (numOfPulses > 64):
            raise DldSimulatorException("List of pulses to large, max. 64")
        
        if (channel == 1):        
            baseAddr = self.ADDR_LIST_GEN1_BASE
        else:
            baseAddr = self.ADDR_LIST_GEN2_BASE
        
        for p in listOfPulses:
            if (not p.validate()):
                raise DldSimulatorException("Invalid pulse configuration")
        
        addrCounter = baseAddr
        for p in listOfPulses:
            self.writeToBus(addrCounter, p.createControlWord())
            addrCounter = addrCounter + 4
            
        if (numOfPulses < 64):
            self.writeToBus(addrCounter, 0x00)
        
        
    def getListGenerator(self, channel):
        if (channel < 1):
            raise DldSimulatorException("Channel number can not be negative")
        if (channel > 2):
            raise DldSimulatorException("Channel number can not be larger than 1")
            
        if (channel == 1):        
            baseAddr = self.ADDR_LIST_GEN1_BASE
        else:
            baseAddr = self.ADDR_LIST_GEN2_BASE
        
        result = []
        currentElementIndex = 0
        while(True):
            if (currentElementIndex >= 64):
                break
            word = self.readFromBus(baseAddr+currentElementIndex*4)
            if (word == 0):
                break
            p = PulseConfig()
            p.readControlWord(word)
            result.append(p)
            currentElementIndex = currentElementIndex + 1
        return result
    
    
    def setListGeneratorActive(self, channel, active):
        if (channel < 1):
            raise DldSimulatorException("Channel number can not be negative")
        if (channel > 2):
            raise DldSimulatorException("Channel number can not be larger than 1")
            
        if (channel == 1):        
            baseAddr = self.ADDR_LIST_GEN1_BASE
        else:
            baseAddr = self.ADDR_LIST_GEN2_BASE
        
        self.writeToBus(baseAddr + 0x100, active)
        
    def isListGeneratorActive(self, channel):
        if (channel < 1):
            raise DldSimulatorException("Channel number can not be negative")
        if (channel > 2):
            raise DldSimulatorException("Channel number can not be larger than 1")
            
        if (channel == 1):        
            baseAddr = self.ADDR_LIST_GEN1_BASE
        else:
            baseAddr = self.ADDR_LIST_GEN2_BASE
            
        return (self.readFromBus(baseAddr + 0x100) == 0x01)
        

In [112]:
p = PulseConfig()

In [84]:
p.set(700, 10, -20, True)

In [85]:
p.createControlWord()
p2 = PulseConfig()
p2.readControlWord(p.createControlWord())
p2.to_string()

't0 = 700  x = 10  y = -20 active = True'

In [72]:
dld = DldSimulatorComm('/dev/ttyUSB1')

In [73]:
dld.setListGenerator(1, [p, p2])

In [74]:
pulses = dld.getListGenerator(1)

In [77]:
pulses[1].to_string()

't0 = 700  x = 10  y = -20 active = True'

In [36]:
setInternalStartInterval(300)

In [167]:
import unittest

class TestDldSim(unittest.TestCase):

    def test_pulse_config(self):
        p = PulseConfig()
        # test set()
        p.set(123,12,-23,True)
        self.assertEqual(p.t0, 123)
        self.assertEqual(p.x ,  12)
        self.assertEqual(p.y , -23)
        self.assertEqual(p.active, True)
        # test validate()
        self.assertEqual(p.validate(), True)
        p.t0 = -1
        self.assertEqual(p.validate(), False)
        p.t0 = 123
        self.assertEqual(p.validate(), True)
        p.x = -33
        self.assertEqual(p.validate(), False)
        p.x = 12
        self.assertEqual(p.validate(), True)
        p.y = 32
        self.assertEqual(p.validate(), False)
        p.y = -23
        self.assertEqual(p.validate(), True)
        
        controlWord = p.createControlWord()
        self.assertEqual(controlWord, 0x49b07b)
        p.set(55,33,-99,True)
        with self.assertRaises(DldSimulatorException):
            p.createControlWord()
            
        p.readControlWord(0x49b07b)
        self.assertEqual(p.t0, 123)
        self.assertEqual(p.x ,  12)
        self.assertEqual(p.y , -23)
        self.assertEqual(p.active, True)
        
    def test_DldSimulator(self):
        tty = glob.glob("/dev/ttyUSB*")[0]
        dld = DldSimulator(tty)
        dld.writeToBus(dld.ADDR_START_CONFIG, 0xdeadbeef)
        self.assertEqual(dld.readFromBus(dld.ADDR_START_CONFIG), 0xdeadbeef)
        
        # test start config
        dld.useExternalStart()
        self.assertEqual(dld.isExternalStart(), True)
        
        dld.useInternalStart()
        self.assertEqual(dld.isExternalStart(), False)
        with self.assertRaises(DldSimulatorException):
            dld.setInternalStartInterval(-1)
        with self.assertRaises(DldSimulatorException):
            dld.setInternalStartInterval(0x10000)
            
        dld.setInternalStartInterval(3456)
        self.assertEqual(dld.getInternalStartInterval(), 3456)
        self.assertEqual(dld.isExternalStart(), False)
        
        # test SpectrumGenerators
        with self.assertRaises(DldSimulatorException):
            dld.setActiveSpectrumGenerators(-1)
        with self.assertRaises(DldSimulatorException):
            dld.setActiveSpectrumGenerators(33)
        dld.setActiveSpectrumGenerators(11)
        self.assertEqual(dld.getActiveSpectrumGenerators(), 11)
        self.assertEqual(dld.readFromBus(dld.ADDR_MULTI_SPECTRUM_GEN), 0b11111111111)
        
        # test ListGenerator
        dld.setListGenerator(1, [])
        self.assertEqual(len(dld.getListGenerator(1)), 0)
        
        p0 = PulseConfig()
        p0.set(123,12,21,True)
        p1 = PulseConfig()
        p1.set(123,12,21,True)
        pf = PulseConfig()
        pf.set(123,456,789,True)
        with self.assertRaises(DldSimulatorException):
            dld.setListGenerator(1, [p0, p1, pf])
            
        # generate list of 64 entries
        pulseList = []
        for i in range(0, 64):
            p = PulseConfig()
            p.set(i*10, i%12, (i%23)-5, True)
            pulseList.append(p)
            
        dld.setListGenerator(1, pulseList)
        readbackList = dld.getListGenerator(1)
        for i in range(0, 64):
            self.assertEqual(pulseList[i].to_string(), readbackList[i].to_string())
            
            
        # generate list of 64 entries
        pulseList = []
        for i in range(0, 10):
            p = PulseConfig()
            p.set(i*9, i%12, (i%23)-5, True)
            pulseList.append(p)
        
        # test, if we write 0 at the end of the list in memory:
        dld.writeToBus(dld.ADDR_LIST_GEN1_BASE + 4*10, 1234)
        dld.setListGenerator(1, pulseList)
        readbackList = dld.getListGenerator(1)
        for i in range(0, 10):
            self.assertEqual(pulseList[i].to_string(), readbackList[i].to_string())
        # check for the 0 at the end in memory
        self.assertEqual(dld.readFromBus(dld.ADDR_LIST_GEN1_BASE + 4*10), 0)
        
        
        # test channel 2:
        # generate list of 64 entries
        pulseList2 = []
        for i in range(0, 4):
            p = PulseConfig()
            p.set(i*3, i%7, (i%33)-17, True)
            pulseList2.append(p)
        dld.setListGenerator(2, pulseList2)
        readbackList2 = dld.getListGenerator(2)
        for i in range(0, 4):
            self.assertEqual(pulseList2[i].to_string(), readbackList2[i].to_string())
            
        # test if generator 1 has not been altered:
        readbackList = dld.getListGenerator(1)
        for i in range(0, 10):
            self.assertEqual(pulseList[i].to_string(), readbackList[i].to_string())
        # check for the 0 at the end in memory
        self.assertEqual(dld.readFromBus(dld.ADDR_LIST_GEN1_BASE + 4*10), 0)
        
        # test activation of the list generator
        dld.setListGeneratorActive(1, True)
        self.assertEqual(dld.readFromBus(dld.ADDR_LIST_GEN1_BASE + 0x100), 0x01)
        self.assertEqual(dld.isListGeneratorActive(1), True)
        
        dld.setListGeneratorActive(1, False)
        self.assertEqual(dld.readFromBus(dld.ADDR_LIST_GEN1_BASE + 0x100), 0x00)
        self.assertEqual(dld.isListGeneratorActive(1), False)
        
        dld.setListGeneratorActive(2, True)
        self.assertEqual(dld.readFromBus(dld.ADDR_LIST_GEN2_BASE + 0x100), 0x01)
        self.assertEqual(dld.isListGeneratorActive(2), True)
        
        dld.setListGeneratorActive(2, False)
        self.assertEqual(dld.readFromBus(dld.ADDR_LIST_GEN2_BASE + 0x100), 0x00)
        self.assertEqual(dld.isListGeneratorActive(2), False)
        
#if __name__ == '__main__':
    #unittest.main()

unittest.main(argv=[''], verbosity=3, exit=False)

test_DldSimulator (__main__.TestDldSim) ... ok
test_pulse_config (__main__.TestDldSim) ... ok

----------------------------------------------------------------------
Ran 2 tests in 3.252s

OK


DldSimulatorException: ('I', 'n', 'v', 'a', 'l', 'i', 'd', ' ', 'p', 'u', 'l', 's', 'e', ' ', 'c', 'o', 'n', 'f', 'i', 'g', 'u', 'r', 'a', 't', 'i', 'o', 'n')

In [60]:
dld.setInternalStartInterval(100)

ValueError: invalid literal for int() with base 16: ''

In [14]:
0xffffffff >> 1

2147483647

In [22]:
bin(8).count("1")

1

In [10]:
dld.setActiveSpectrumGenerators(10)

In [11]:
dld.getActiveSpectrumGenerators()

10

In [57]:
0x1_2_3

291

In [61]:
del dld